# Implementation of the Kuribayashi BERT minus model

## libraries

In [1]:
!pip install transformers --upgrade
!pip install ipywidgets
!pip install IProgress
!pip install datasets
!pip install torch-lr-finder

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import transformers
from transformers import BertTokenizer, BertConfig
from transformers import BertModel, BertForSequenceClassification
from transformers import BatchEncoding, default_data_collator, DataCollatorWithPadding

import torch
import torch.nn as nn

import numpy as np

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

import datasets

from torch.utils.data import DataLoader

from tqdm import tqdm
from operator import itemgetter

In [3]:
print(transformers.__version__)

4.25.1


## tokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [5]:
tokenizer

PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

## data

In [6]:
# DATA_FOLDER = '/notebooks/Data/bert_sequence_classification'
DATA_FILE = '/notebooks/KURI-BERT/notebooks/full_formula_w_fts/pe_dataset_for_bert_minus_w_fts.pt'
RESULTS_FOLDER = '/notebooks/KURI-BERT/results'

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
device

device(type='cuda')

## Load data

In [9]:
dataset = torch.load(DATA_FILE)

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['paragraph', 'paragraph_components_list', 'paragraph_labels_list', 'paragraph_markers_list', 'split', 'essay_nr', 'paragraph_fts_as_txt_list', 'paragraph_labels', 'paragraph_ac_spans', 'paragraph_am_spans', 'paragraph_w_fts_as_txt', 'ac_spans_new', 'sanity_new', 'am_spans_new', 'feature_spans_new'],
        num_rows: 1088
    })
    test: Dataset({
        features: ['paragraph', 'paragraph_components_list', 'paragraph_labels_list', 'paragraph_markers_list', 'split', 'essay_nr', 'paragraph_fts_as_txt_list', 'paragraph_labels', 'paragraph_ac_spans', 'paragraph_am_spans', 'paragraph_w_fts_as_txt', 'ac_spans_new', 'sanity_new', 'am_spans_new', 'feature_spans_new'],
        num_rows: 358
    })
    validation: Dataset({
        features: ['paragraph', 'paragraph_components_list', 'paragraph_labels_list', 'paragraph_markers_list', 'split', 'essay_nr', 'paragraph_fts_as_txt_list', 'paragraph_labels', 'paragraph_ac_spans', 'paragraph_am_sp

In [11]:
dataset['train']['feature_spans_new']

[[[23, 50], [75, 102], [142, 169]],
 [[47, 74], [111, 138], [159, 186]],
 [[25, 52], [72, 99], [132, 159]],
 [[21, 48], [85, 112]],
 [[14, 41], [75, 102], [118, 145], [158, 185]],
 [[9, 36],
  [51, 78],
  [97, 124],
  [138, 165],
  [177, 204],
  [228, 255],
  [280, 307]],
 [[23, 50], [60, 87], [102, 129], [163, 190]],
 [[43, 70]],
 [[22, 49], [68, 95]],
 [[19, 46], [76, 103], [116, 143]],
 [[25, 52], [78, 105], [130, 157], [174, 201]],
 [[21, 48], [62, 89], [105, 132], [145, 172], [192, 219]],
 [[31, 58]],
 [[61, 88]],
 [[31, 58], [84, 111], [145, 172], [206, 233]],
 [[19, 46], [65, 92], [116, 143], [163, 190], [210, 237]],
 [[81, 108], [130, 157]],
 [[32, 59], [69, 96]],
 [[23, 50], [64, 91], [110, 137], [155, 182], [201, 228]],
 [[16, 43], [59, 86], [111, 138], [152, 179], [195, 222], [244, 271]],
 [[18, 45], [70, 97], [113, 140], [165, 192]],
 [[16, 43], [53, 80], [101, 128], [145, 172], [199, 226], [251, 278]],
 [[19, 46],
  [71, 98],
  [127, 154],
  [174, 201],
  [230, 257],
  [28

### preprocessing

In [12]:
MAX_LENGTH = 0
MAX_SPAN = 0

for split in ['train', 'test', 'validation']:
    
    for col_name in ['am_spans_new', 'ac_spans_new', 'feature_spans_new']:
        
        for x in dataset[split][col_name]:
            
            if max(x,key=itemgetter(1))[1] > MAX_SPAN:
                
                MAX_SPAN = max(x,key=itemgetter(1))[1]
                MAX_SPAN = min(MAX_SPAN, tokenizer.model_max_length - 1)
            
            if len(x) > MAX_LENGTH:
                
                MAX_LENGTH = len(x)

In [13]:
MAX_SPAN

511

In [14]:
def get_padding(batch, padding_target):    
    
    if padding_target == 'am_spans':
        
        col_name = 'am_spans_new'
        padding_val = [[-1,-1]]
        max_length = MAX_LENGTH
        
    elif padding_target == 'ac_spans':
        
        col_name = 'ac_spans_new'
        padding_val = [[-1,-1]]
        max_length = MAX_LENGTH
        
    elif padding_target == 'fts_spans':
        
        col_name = 'feature_spans_new'
        padding_val = [[-1,-1]]
        max_length = MAX_LENGTH
        
    elif padding_target == 'label':
    
        col_name = 'paragraph_labels'
        padding_val = [-100] # -1 previously       
        max_length = MAX_LENGTH # max([len(l) for l in batch[col_name]]) # cause some batch had 4 x 10

    padded_spans = []

    for idx, span in enumerate(batch[col_name]):

        padded_span = batch[col_name][idx] + (max_length - len(span)) * padding_val
        padded_spans.append(padded_span)

    return padded_spans         

In [15]:
def get_combined_spans(am_spans_ll, ac_spans_ll, fts_spans_ll):
    
    spans_ll = []
    
    for am_spans, ac_spans, fts_spans in zip(am_spans_ll, ac_spans_ll, fts_spans_ll):
        
        spans = []
        
        for am_span, ac_span, fts_span in zip(am_spans, ac_spans, fts_spans):

            span = [am_span, ac_span, fts_span]
            spans.extend(span)
            
        spans_ll.append(spans)

    return spans_ll

### tokenize 

In [16]:
# max_length = 200 for use in the max_length in the tokenizer so that the things are of equal dim.

In [17]:
def tokenize(batch):
    
    tokenized_text = tokenizer(batch['paragraph_w_fts_as_txt'], truncation=True, padding=True, max_length=512)
    tokenized_text['label'] = get_padding(batch, 'label')
    tokenized_text['am_spans'] = get_padding(batch, 'am_spans')
    tokenized_text['ac_spans'] = get_padding(batch, 'ac_spans')
    tokenized_text['fts_spans'] = get_padding(batch, 'fts_spans')
    tokenized_text['spans'] = get_combined_spans(tokenized_text['am_spans'], tokenized_text['ac_spans'], tokenized_text['fts_spans'])      
    
    return tokenized_text

In [18]:
dataset = dataset.map(tokenize, batched=True, batch_size=len(dataset['train']))

Parameter 'function'=<function tokenize at 0x7f24a4233a60> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ac_spans', 'ac_spans_new', 'am_spans', 'am_spans_new', 'attention_mask', 'essay_nr', 'feature_spans_new', 'fts_spans', 'input_ids', 'label', 'paragraph', 'paragraph_ac_spans', 'paragraph_am_spans', 'paragraph_components_list', 'paragraph_fts_as_txt_list', 'paragraph_labels', 'paragraph_labels_list', 'paragraph_markers_list', 'paragraph_w_fts_as_txt', 'sanity_new', 'spans', 'split', 'token_type_ids'],
        num_rows: 1088
    })
    test: Dataset({
        features: ['ac_spans', 'ac_spans_new', 'am_spans', 'am_spans_new', 'attention_mask', 'essay_nr', 'feature_spans_new', 'fts_spans', 'input_ids', 'label', 'paragraph', 'paragraph_ac_spans', 'paragraph_am_spans', 'paragraph_components_list', 'paragraph_fts_as_txt_list', 'paragraph_labels', 'paragraph_labels_list', 'paragraph_markers_list', 'paragraph_w_fts_as_txt', 'sanity_new', 'spans', 'split', 'token_type_ids'],
        num_rows: 358
    })
    validation: Dataset({
        feat

In [20]:
dataset['train'][0]['spans']

[[0, 0],
 [2, 21],
 [23, 50],
 [-1, -1],
 [57, 73],
 [75, 102],
 [-1, -1],
 [105, 140],
 [142, 169],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1],
 [-1, -1]]

In [21]:
len(dataset['train'][0]['spans'])

36

In [22]:
dataset['train'][1]['label']

[2, 2, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100]

In [23]:
dataset['train'].features

{'ac_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'ac_spans_new': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'am_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'am_spans_new': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'essay_nr': Value(dtype='string', id=None),
 'feature_spans_new': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'fts_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': Sequence(feature=Value(dtype='int64',

In [24]:
dataset['test'].features

{'ac_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'ac_spans_new': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'am_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'am_spans_new': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'essay_nr': Value(dtype='string', id=None),
 'feature_spans_new': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'fts_spans': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': Sequence(feature=Value(dtype='int64',

In [25]:
dataset['test'].features['spans'] = datasets.Array2D(shape=(36, 2), dtype="int32")
dataset['train'].features['spans'] = datasets.Array2D(shape=(36, 2), dtype="int32")
dataset['validation'].features['spans'] = datasets.Array2D(shape=(36, 2), dtype="int32")

In [26]:
dataset = dataset.map(lambda batch: batch, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [27]:
dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'spans', 'label'])

## span representation function

In [28]:
def minus_one(t):
    
    return torch.where(t == 0, 0, t-1)

def plus_one(t):
    
    return torch.where(t == MAX_SPAN, MAX_SPAN, t+1) # changed from t == MAX_SPAN

In [29]:
def get_span_representations(outputs, spans):
    
    #print('spans shape', spans.shape)
    
    batch_size = spans.shape[0]
    
    nr_span_indices = spans.shape[1]
    
    #print('nr span indices', nr_span_indices)
    
    # nr_span_indices = 24 # xxx. hardcode just to check
    
    idx_l_ams = range(0, nr_span_indices, 3) # [0,2,4,6 etc]
    idx_l_acs = range(1, nr_span_indices, 3) # [1,3,5,7 etc]
    idx_l_fts = range(2, nr_span_indices, 3)
    
    am_spans = spans[:, idx_l_ams, :] + 1 # adds 1 to all span indices (both am and ac) to offset for the CLS token in the input_ids.
    ac_spans = spans[:, idx_l_acs, :] + 1
    fts_spans = spans[:, idx_l_fts, :] + 1
    
    
    # fix for [-1, +1] problem
    
    am_spans_minus_one = minus_one(am_spans) # xxx. added to solve bug 2
    am_spans_plus_one = plus_one(am_spans) # xxx. added to solve bug 2
    
    ac_spans_minus_one = minus_one(ac_spans) # xxx. added to solve bug 2
    ac_spans_plus_one = plus_one(ac_spans) # xxx. added to solve bug 2
    
    fts_spans_minus_one = minus_one(fts_spans) # xxx. added to solve bug 2
    fts_spans_plus_one = plus_one(fts_spans) # xxx. added to solve bug 2
    
    
    am_spans = am_spans.flatten(start_dim=1)
    ac_spans = ac_spans.flatten(start_dim=1)
    fts_spans = fts_spans.flatten(start_dim=1)
    
    nr_adus = ac_spans.shape[1] // 2
    
    
    am_spans_minus_one = am_spans_minus_one.flatten(start_dim=1) # xxx. added to solve bug 2
    am_spans_plus_one = am_spans_plus_one.flatten(start_dim=1) # xxx. added to solve bug 2
    
    ac_spans_minus_one = ac_spans_minus_one.flatten(start_dim=1) # xxx. added to solve bug 2
    ac_spans_plus_one = ac_spans_plus_one.flatten(start_dim=1) # xxx. added to solve bug 2
    
    fts_spans_minus_one = fts_spans_minus_one.flatten(start_dim=1) # xxx. added to solve bug 2
    fts_spans_plus_one = fts_spans_plus_one.flatten(start_dim=1) # xxx. added to solve bug 2
    
    ############# FOR AMs #################
    
    outputs_am = outputs[:,am_spans,:]
    #print('outputs am juste directly from outputs:', outputs_am.shape)
    outputs_am = torch.cat([outputs_am[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_am = outputs_am.reshape(batch_size, nr_adus * 2, -1)
    
    #print('outputs_ am after reshape:', outputs_am.shape)
    
    
    
    outputs_am_minus_one = outputs[:,am_spans_minus_one,:] # xxx. added to solve bug 2
    outputs_am_minus_one = torch.cat([outputs_am_minus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_am_minus_one = outputs_am_minus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    outputs_am_plus_one = outputs[:,am_spans_plus_one,:] # xxx. added to solve bug 2
    outputs_am_plus_one = torch.cat([outputs_am_plus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_am_plus_one = outputs_am_plus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    
    
    ### Now that we have outputs_am i.e. outputs at am_span indices, now create the four Kuri forumlas for AMs
    
    # ============== the corrected 1st one =================== 
    
    outputs_am_first_term = torch.cat([outputs_am[:,i+1,:] - outputs_am_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1) # i + 1 here means j in kuri, i here means i in kuri
    outputs_am_first_term = outputs_am_first_term.reshape(batch_size, -1, 768)
    
     # ============== the corrected 2nd one ==================
    
    outputs_am_second_term = torch.cat([outputs_am[:,i,:] - outputs_am_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run # changed from +1 to +2 to ensure +2 is not a problem for AMs
    outputs_am_second_term = outputs_am_second_term.reshape(batch_size, -1, 768)
    
    
        # ============== the corrected third one ================== 
    
    outputs_am_third_term = torch.cat([outputs_am_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_am_third_term = outputs_am_third_term.reshape(batch_size, -1, 768)
    

        # ============== the corrected fourth one ==================
    
    outputs_am_fourth_term = torch.cat([outputs_am_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run # changed from +1 to +2 to ensure +2 is not a problem for AMs
    outputs_am_fourth_term = outputs_am_fourth_term.reshape(batch_size, -1, 768)
    
    # ============== NOW CONCATENATE THEM =========
    
    
    am_minus_representations = torch.cat([outputs_am_first_term, outputs_am_second_term, outputs_am_third_term, outputs_am_fourth_term], dim=-1)   
    
    
    
    ### am minus span representation according to kuribayashi paper is now above.
    
    
    outputs_ac = outputs[:,ac_spans,:]
    outputs_ac = torch.cat([outputs_ac[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_ac = outputs_ac.reshape(batch_size, nr_adus * 2, -1)
    
    
    outputs_ac_minus_one = outputs[:,ac_spans_minus_one,:] # xxx. added to solve bug 2
    outputs_ac_minus_one = torch.cat([outputs_ac_minus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_ac_minus_one = outputs_ac_minus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    outputs_ac_plus_one = outputs[:,ac_spans_plus_one,:] # xxx. added to solve bug 2
    outputs_ac_plus_one = torch.cat([outputs_ac_plus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_ac_plus_one = outputs_ac_plus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    ### Now that we have outputs_ac i.e. outputs at ac_span indices, now create the four Kuri forumlas for ACs
    
    
    # ============== the corrected first one ===================
    
    outputs_ac_first_term = torch.cat([outputs_ac[:,i+1,:] - outputs_ac_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_ac_first_term = outputs_ac_first_term.reshape(batch_size, -1, 768)
    
        # ============== the correct second one ==================
    
    outputs_ac_second_term = torch.cat([outputs_ac[:,i,:] - outputs_ac_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run
    outputs_ac_second_term = outputs_ac_second_term.reshape(batch_size, -1, 768)
    
    
    # ============== the corrected third term ==================
    
    outputs_ac_third_term = torch.cat([outputs_ac_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_ac_third_term = outputs_ac_third_term.reshape(batch_size, -1, 768)

    
     # ============== the corrected fourth term ================== xxx. for .788
    
    outputs_ac_fourth_term = torch.cat([outputs_ac_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1) # changed + 2 to + 1 to make it run
    outputs_ac_fourth_term = outputs_ac_fourth_term.reshape(batch_size, -1, 768)
    
    # ============== NOW CONCATENATE THEM =========
    
    
    ac_minus_representations = torch.cat([outputs_ac_first_term, outputs_ac_second_term, outputs_ac_third_term, outputs_ac_fourth_term], dim=-1)   
    
    
    
    ### ac minus span representation according to kuribayashi paper is now above.
    
    
     ############# FOR Fts #################
    
    outputs_fts = outputs[:,am_spans,:] # am spans for checking
    #print('outputs fts juste directly from the outputs:', outputs_fts.shape)
    outputs_fts = torch.cat([outputs_fts[i,i,:,:] for i in range(batch_size)], dim=0)
    outputs_fts = outputs_fts.reshape(batch_size, nr_adus * 2, -1)
    
    #print('output fts after reshape:', outputs_fts.shape)
    
    
    
    outputs_fts_minus_one = outputs[:,fts_spans_minus_one,:] # xxx. added to solve bug 2
    outputs_fts_minus_one = torch.cat([outputs_fts_minus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_fts_minus_one = outputs_fts_minus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    outputs_fts_plus_one = outputs[:,fts_spans_plus_one,:] # xxx. added to solve bug 2
    outputs_fts_plus_one = torch.cat([outputs_fts_plus_one[i,i,:,:] for i in range(batch_size)], dim=0) # xxx. added to solve bug 2
    outputs_fts_plus_one = outputs_fts_plus_one.reshape(batch_size, nr_adus * 2, -1) # xxx. added to solve bug 2
    
    #print('outputs_fts_plus_one', outputs_fts_plus_one.shape)
    
    
    
    ### Now that we have outputs_am i.e. outputs at am_span indices, now create the four Kuri forumlas for AMs
    
    # ============== (batch_size x nr_adus x 768) =================== 
    
    outputs_fts_first_term = torch.cat([outputs_fts[:,i+1,:] - outputs_fts_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1) # i + 1 here means j in kuri, i here means i in kuri
    outputs_fts_first_term = outputs_fts_first_term.reshape(batch_size, -1, 768)
    #print('outputs_fts_first_term:', outputs_fts_first_term.shape)
    
     # ============== the corrected 2nd one ==================
    
    outputs_fts_second_term = torch.cat([outputs_fts[:,i,:] - outputs_fts_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_fts_second_term = outputs_fts_second_term.reshape(batch_size, -1, 768)
    #print('outputs_fts_second_term:', outputs_fts_second_term.shape)
    
        # ============== the corrected third one ================== 
    
    outputs_fts_third_term = torch.cat([outputs_fts_minus_one[:,i,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_fts_third_term = outputs_fts_third_term.reshape(batch_size, -1, 768)
    

        # ============== the corrected fourth one ==================
    
    outputs_fts_fourth_term = torch.cat([outputs_fts_plus_one[:,i+1,:] for i in range(0, nr_adus * 2, 2)], dim=1)
    outputs_fts_fourth_term = outputs_fts_fourth_term.reshape(batch_size, -1, 768)
    #print('outputs_fts_fourth_term:', outputs_fts_fourth_term.shape)
    
    # ============== NOW CONCATENATE THEM =========
    
    
    fts_minus_representations = torch.cat([outputs_fts_first_term, outputs_fts_second_term, outputs_fts_third_term, outputs_fts_fourth_term], dim=-1)   
    
    
    
    ### fts minus span representation according to kuribayashi paper is now above.
    
    print('am rep final:', am_minus_representations.shape)
    print('ac rep final:', ac_minus_representations.shape)
    print('fts rep final:', fts_minus_representations.shape)
    

    
    return am_minus_representations, ac_minus_representations, fts_minus_representations                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

### span representation function old

In [30]:
# def get_span_representations(outputs, spans):
    
#     # print(spans.shape)
    
#     batch_size = spans.shape[0]
#     nr_span_indices = spans.shape[1]
    
#     print('nr span indices: ', nr_span_indices)
    
#     idx_l_ams = range(0, nr_span_indices, 3) # [0,2,4,6 etc]
#     # print('idx am', idx_l_ams)
#     idx_l_acs = range(1, nr_span_indices, 3) # [1,3,5,7 etc]
#     #print('idx ac', idx_l_acs)
#     idx_l_fts = range(2, nr_span_indices, 3) # xxx. added for fts task.
#     #print('idx fts', idx_l_fts)
    
    
#     am_spans = spans[:, idx_l_ams, :] + 1 # adds 1 to all span indices (both am and ac) to offset for the CLS token in the input_ids.
    
#     ac_spans = spans[:, idx_l_acs, :] + 1    
    
#     fts_spans = spans[:, idx_l_fts, :] + 1 ### xxx. 
    
    
#     am_spans = am_spans.flatten(start_dim=1)
#     print('am spans:', am_spans.shape)
#     ac_spans = ac_spans.flatten(start_dim=1)
#     print('ac spans:', ac_spans.shape)
#     fts_spans = fts_spans.flatten(start_dim=1) ### xxx
#     print('fts spans:', fts_spans.shape)
    
#     outputs_am = outputs[:,am_spans,:]
#     print('outputs am before cat:', outputs_am.shape)
#     outputs_am = torch.cat([outputs_am[i,i,:,:] for i in range(batch_size)], dim=0)
#     print('outputs am after cat:', outputs_am.shape)
#     outputs_am = outputs_am.reshape(batch_size, nr_span_indices, -1)
#     print('outputs am after reshape:', outputs_am.shape)
    
#     ### Now that we have outputs_am i.e. outputs at am_span indices, now create the four Kuri forumlas for AMs
    
#     # ============== FIRST TERM ===================
    
#     outputs_am_first_term = torch.cat([outputs_am[:,i+1,:] - outputs_am[:,i-1,:] for i in range(0, nr_span_indices, 3)], dim=1) 
#     outputs_am_first_term = outputs_am_first_term.reshape(batch_size, -1, 768)
    
#     # ============== SECOND TERM ==================
    
#     outputs_am_second_term = torch.cat([outputs_am[:,i,:] - outputs_am[:,i+1,:] for i in range(0, nr_span_indices, 3)], dim=1) # changed + 2 to + 1 to make it run # changed from +1 to +2 to ensure +2 is not a problem for AMs
#     outputs_am_second_term = outputs_am_second_term.reshape(batch_size, -1, 768)
    
    
#     # ============== THIRD TERM ==================
    
#     outputs_am_third_term = torch.cat([outputs_am[:,i-1,:] for i in range(0, nr_span_indices, 3)], dim=1)
#     outputs_am_third_term = outputs_am_third_term.reshape(batch_size, -1, 768)
    
#     # ============== FOURTH TERM ==================
    
#     outputs_am_fourth_term = torch.cat([outputs_am[:,i+1,:] for i in range(0, nr_span_indices, 3)], dim=1) # changed + 2 to + 1 to make it run # changed from +1 to +2 to ensure +2 is not a problem for AMs
#     outputs_am_fourth_term = outputs_am_fourth_term.reshape(batch_size, -1, 768)
    
#     # ============== NOW CONCATENATE THEM =========
    
    
#     am_minus_representations = torch.cat([outputs_am_first_term, outputs_am_second_term, outputs_am_third_term, outputs_am_fourth_term], dim=-1)  
    
#     print('am reps apres:', am_minus_representations.shape)
    
    
    
#     ### am minus span representation according to kuribayashi paper is now here.
    

    
#     outputs_ac = outputs[:,ac_spans,:]
#     # print('outputs ac shape:', outputs_ac.shape)
#     outputs_ac = torch.cat([outputs_ac[i,i,:,:] for i in range(batch_size)], dim=0)
    
#     outputs_ac = outputs_ac.reshape(batch_size, nr_span_indices, -1)
    
#     ### Now that we have outputs_ac i.e. outputs at ac_span indices, now create the four Kuri forumlas for ACs    
    
#     # ============== FIRST TERM ===================
    
#     outputs_ac_first_term = torch.cat([outputs_ac[:,i+1,:] - outputs_ac[:,i-1,:] for i in range(0, nr_span_indices, 3)], dim=1)
#     outputs_ac_first_term = outputs_ac_first_term.reshape(batch_size, -1, 768)
    
#     # ============== SECOND TERM ==================
    
#     outputs_ac_second_term = torch.cat([outputs_ac[:,i,:] - outputs_ac[:,i+1,:] for i in range(0, nr_span_indices, 3)], dim=1) # changed + 2 to + 1 to make it run
#     outputs_ac_second_term = outputs_ac_second_term.reshape(batch_size, -1, 768)
    
    
#     # ============== THIRD TERM ==================
    
#     outputs_ac_third_term = torch.cat([outputs_ac[:,i-1,:] for i in range(0, nr_span_indices, 3)], dim=1)
#     outputs_ac_third_term = outputs_ac_third_term.reshape(batch_size, -1, 768)
    
#     # ============== FOURTH TERM ==================
    
#     outputs_ac_fourth_term = torch.cat([outputs_ac[:,i+1,:] for i in range(0, nr_span_indices, 3)], dim=1) # changed + 2 to + 1 to make it run
#     outputs_ac_fourth_term = outputs_ac_fourth_term.reshape(batch_size, -1, 768)
    
#     # ============== NOW CONCATENATE THEM =========
    
    
#     ac_minus_representations = torch.cat([outputs_ac_first_term, outputs_ac_second_term, outputs_ac_third_term, outputs_ac_fourth_term], dim=-1)   
    
    
    
#     ### ac minus span representation according to kuribayashi paper is now here.
    
#     ### Now that we have outputs_ac i.e. outputs at ac_span indices, now create the four Kuri forumlas for Features as Text 
    
#     outputs_fts = outputs[:,fts_spans,:]
#     #print('outputs fts shape:', outputs_fts.shape)
#     outputs_fts = torch.cat([outputs_fts[i,i,:,:] for i in range(batch_size)], dim=0)
    
#     outputs_fts = outputs_fts.reshape(batch_size, nr_span_indices, -1)
    
#     ### Now that we have outputs_ac i.e. outputs at ac_span indices, now create the four Kuri forumlas for ACs    
    
#     # ============== FIRST TERM ===================
    
#     outputs_fts_first_term = torch.cat([outputs_fts[:,i+1,:] - outputs_fts[:,i-1,:] for i in range(0, nr_span_indices, 3)], dim=1)
#     outputs_fts_first_term = outputs_fts_first_term.reshape(batch_size, -1, 768)
    
#     # ============== SECOND TERM ==================
    
#     outputs_fts_second_term = torch.cat([outputs_fts[:,i,:] - outputs_fts[:,i+1,:] for i in range(0, nr_span_indices, 3)], dim=1) # changed + 2 to + 1 to make it run
#     outputs_fts_second_term = outputs_fts_second_term.reshape(batch_size, -1, 768)
    
    
#     # ============== THIRD TERM ==================
    
#     outputs_fts_third_term = torch.cat([outputs_fts[:,i-1,:] for i in range(0, nr_span_indices, 3)], dim=1)
#     outputs_fts_third_term = outputs_fts_third_term.reshape(batch_size, -1, 768)
    
#     # ============== FOURTH TERM ==================
    
#     outputs_fts_fourth_term = torch.cat([outputs_fts[:,i+1,:] for i in range(0, nr_span_indices, 3)], dim=1) # changed + 2 to + 1 to make it run
#     outputs_fts_fourth_term = outputs_fts_fourth_term.reshape(batch_size, -1, 768)
    
#     # ============== NOW CONCATENATE THEM =========
    
    
#     fts_minus_representations = torch.cat([outputs_fts_first_term, outputs_fts_second_term, outputs_fts_third_term, outputs_fts_fourth_term], dim=-1)
    

    
#     return am_minus_representations, ac_minus_representations, fts_minus_representations                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

## custom BERT model

In [31]:
class CustomBERTKuri(nn.Module):

    def __init__(self, first_model, model_am, model_ac, model_fts, nr_classes):
        
        super(CustomBERTKuri, self).__init__()
        
        self.first_model = first_model
        
        self.intermediate_linear_am = nn.Linear(3072, 768)
        self.intermediate_linear_ac = nn.Linear(3072, 768) 
        self.intermediate_linear_fts = nn.Linear(3072, 768)
        
        self.model_am = model_am
        self.model_ac = model_ac
        self.model_fts = model_fts
        
        self.nr_classes = nr_classes
                
        self.fc = nn.Linear(self.model_am.config.hidden_size + self.model_ac.config.hidden_size + self.model_fts.config.hidden_size, self.nr_classes)
        # self.fc = nn.Linear(self.model_am.config.hidden_size + self.model_ac.config.hidden_size, self.nr_classes) 

    def forward(self, inputs):
        
        batch_tokenized, batch_spans = inputs 
        outputs = self.first_model(batch_tokenized)[0] # ** removed to correct error.
        # spans = batch_spans # remove this spans thing cause we are now giving it just the spans themselves.
        am_minus_representations, ac_minus_representations, fts_minus_representations = get_span_representations(outputs, batch_spans)
        print('passed and gotten reps for all three successfully!')
        print('am rep final in model:', am_minus_representations.shape)
        print('ac rep final in model:', ac_minus_representations.shape)
        print('fts rep final in model:', fts_minus_representations.shape)
        #am_minus_representations, ac_minus_representations = get_span_representations(outputs, batch_spans)
        
        #print('am minus rep:', am_minus_representations.shape)
        #print('ac minus rep:', ac_minus_representations.shape)
        #print('fts minus rep:', fts_minus_representations.shape)

        am_minus_representations = self.intermediate_linear_am(am_minus_representations)
        print('passed first linear')
        ac_minus_representations = self.intermediate_linear_ac(ac_minus_representations)
        print('passed second linear')
        fts_minus_representations = self.intermediate_linear_fts(fts_minus_representations)
        print('passed third linear')
        
        #print('am minus rep:', am_minus_representations.shape)
        #print('ac minus rep:', ac_minus_representations.shape)
        #print('fts minus rep:', fts_minus_representations.shape)

        output_model_am = self.model_am(inputs_embeds = am_minus_representations)[0]
        output_model_ac = self.model_ac(inputs_embeds = ac_minus_representations)[0]
        output_model_fts = self.model_fts(inputs_embeds = fts_minus_representations)[0]

        adu_representations = torch.cat([output_model_am, output_model_ac, output_model_fts], dim=-1)
        #adu_representations = torch.cat([output_model_am, output_model_ac], dim=-1)
        # print("adu rep:", adu_representations.shape)
        output = self.fc(adu_representations)
        # print("model class output avant reshape:", output.shape)
        # output = output.reshape(-1, self.nr_classes)
        # print("model class output apres:", output.shape)
        return output

## Run

In [32]:
NB_EPOCHS = 1
BATCH_SIZE = 4

In [33]:
first_model = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [34]:
model_am = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [35]:
model_ac = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [36]:
model_fts = BertModel(BertConfig.from_pretrained("bert-base-uncased"))

In [37]:
custom_model = CustomBERTKuri(first_model, model_am, model_ac, model_fts, 3)

In [38]:
custom_model.to(device)

CustomBERTKuri(
  (first_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [39]:
loss = nn.CrossEntropyLoss(ignore_index=- 100)

In [40]:
optimizer = torch.optim.AdamW(custom_model.parameters(), lr=1.8738174228603844e-05)

In [41]:
# 1.8738174228603844e-05
# best learning rate found by the whole leslie business
# new best LR found= 9.999999999999997e-06

In [42]:
NR_BATCHES = len(dataset['train']) / BATCH_SIZE
num_training_steps = NB_EPOCHS * NR_BATCHES
num_warmup_steps = int(0.2 * num_training_steps)

In [43]:
def lr_lambda(current_step: int):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(
            0.0, float(num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps))
        )

    # return LambdaLR(optimizer, lr_lambda, last_epoch)

In [44]:
# commented for LR Finder. remove it from optimizer.
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

### create dataloaders

In [45]:
train_dataloader = DataLoader(dataset['train'], batch_size=BATCH_SIZE, shuffle=False) # xxx.
val_dataloader = DataLoader(dataset['validation'], batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset['test'], batch_size=BATCH_SIZE, shuffle=True)

In [46]:
def flatten_list(list_of_lists):
    return [x for sublist in list_of_lists for x in sublist]

In [47]:
def remove_dummy_labels(test_preds, test_labels):
    
    idxes = []
    test_labels_l = []
    for idx, val in enumerate(test_labels):
        if val != -100:
            idxes.append(idx)
            test_labels_l.append(val)
    
    test_preds_l = []
    for idx, val in enumerate(test_preds):
        for good_idx in idxes:
            if idx == good_idx:
                test_preds_l.append(val)
        
    return test_preds_l, test_labels_l

In [48]:
for i, b in enumerate(train_dataloader):
    if i == 40:
        break

In [49]:
b

{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[ 101, 1999, 7091,  ...,    0,    0,    0],
         [ 101, 2000, 4088,  ..., 2256, 2147,  102],
         [ 101, 4914, 2135,  ...,    0,    0,    0],
         [ 101, 2009, 2003,  ...,    0,    0,    0]]),
 'label': tensor([[   0,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100],
         [   1,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2],
         [   1,    2,    2,    2, -100, -100, -100, -100, -100, -100, -100, -100],
         [   1,    0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]]),
 'spans': tensor([[[  0,   1],
          [  3,  15],
          [ 17,  44],
          [ -1,  -1],
          [ 48,  52],
          [ 54,  81],
          [ -1,  -1],
          [ 84,  99],
          [101, 128],
          [ -1,  -1],
          [ -1,  -1],
          [ -1,  -1

### training loop

In [50]:
from transformers import Trainer, TrainingArguments
from datasets import load_metric

In [51]:
import random
from torch_lr_finder import LRFinder

In [52]:
# def compute_loss(self, model, inputs, return_outputs=False):

#     labels = inputs["labels"]
#     labels = labels.flatten() # xxx.



#     outputs = model(inputs['input_ids'], inputs['spans'])
#     outputs = outputs.flatten(0,1) # xxx. for the 4 x 12 , 3 output of the main class.


#     loss_fct = nn.CrossEntropyLoss()#(weight=class_weights)
#     # loss = loss_fct(outputs, labels.flatten())
#     loss = loss_fct(outputs, labels) # xxx
#     #print("loss:", loss)
#     return (loss, outputs) if return_outputs else loss

In [53]:
# metric = load_metric('f1')

# def compute_metrics(eval_pred):

#     logits, labels = eval_pred

    
#     print("logits", logits.shape)    
#     predictions = np.argmax(logits, axis=-1)
    
#     print("preds:", predictions.shape)
    
#     return metric.compute(predictions=predictions, references=labels, average='macro')

### LR Finder Leslie Smith 

## training 

In [54]:
def train(model, loss=None, optimizer=None, train_dataloader=None, val_dataloader=None, nb_epochs=20):
    """Training loop"""

    min_f1 = -torch.inf
    train_losses = []
    val_losses = []

    # Iterrate over epochs
    for e in range(nb_epochs):

        # Training
        train_loss = 0.0

        for i, batch in enumerate(tqdm(train_dataloader)):            
            
            print(i)
            # unpack batch             
            labels = batch['label'].to(device)
            spans = batch['spans'].to(device)
            input_ids = batch['input_ids'].to(device)
            
            inputs = input_ids, spans
            
            # Reset gradients to 0
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(inputs)
            
            # Compute training loss
            current_loss = loss(outputs.flatten(0,1), labels.flatten())
            train_loss += current_loss.detach().item()

            # Compute gradients
            current_loss.backward()

            # Update weights
            optimizer.step()            
            
            del batch
        
        scheduler.step()
            
        
        # Validation
        val_loss = 0.0

        # Put model in eval mode
        model.eval()
        
        preds_l = []
        labels_l = []
        
        for batch in tqdm(val_dataloader):            
            
            # unpack batch             
            labels = batch['label'].to(device)
            spans = batch['spans'].to(device)
            input_ids = batch['input_ids'].to(device)
            
            inputs = input_ids, spans
            
            # Forward Pass
            outputs = model(inputs)

            # Compute validation loss
            current_loss = loss(outputs.flatten(0,1), labels.flatten())
            val_loss += current_loss.detach().item()
            
            preds_for_f1 = torch.argmax(outputs, dim=2).flatten().tolist()
            labels_for_f1 = labels.flatten().tolist()
            
            preds_l.append(preds_for_f1)
            labels_l.append(labels_for_f1)
            
            del batch
        
        # Prints
        
        preds_l = flatten_list(preds_l)
        labels_l = flatten_list(labels_l)
        
        preds_l, labels_l = remove_dummy_labels(preds_l, labels_l)
        
        f1_score_epoch = f1_score(preds_l, labels_l, average='macro')        
        
        print(f"Epoch {e+1}/{nb_epochs} \
                \t Training Loss: {train_loss/len(train_dataloader):.3f} \
                \t Validation Loss: {val_loss/len(val_dataloader):.3f} \
                \t F1 score: {f1_score_epoch}")
        
        train_losses.append(train_loss/len(train_dataloader))
        val_losses.append(val_loss/len(val_dataloader))
        

        # Save model if val loss decreases
        if f1_score_epoch > min_f1:

            min_f1 = f1_score_epoch
            torch.save(model.first_model.state_dict(), 'first_model.pt')
            torch.save(model.model_am.state_dict(), 'model_am.pt')
            torch.save(model.model_ac.state_dict(), 'model_ac.pt')
            torch.save(model.model_fts.state_dict(), 'model_fts.pt')
            torch.save(model.state_dict(), 'best_model.pt')
            
    return train_losses, val_losses

In [55]:
train_losses, val_losses = train(custom_model, loss, optimizer, train_dataloader, val_dataloader, NB_EPOCHS)

  0%|          | 0/272 [00:00<?, ?it/s]

0
am rep final: torch.Size([4, 12, 3072])
ac rep final: torch.Size([4, 12, 3072])
fts rep final: torch.Size([4, 12, 3072])
passed and gotten reps for all three successfully!
am rep final in model: torch.Size([4, 12, 3072])
ac rep final in model: torch.Size([4, 12, 3072])
fts rep final in model: torch.Size([4, 12, 3072])
passed first linear
passed second linear
passed third linear


  0%|          | 1/272 [00:00<04:10,  1.08it/s]

1
am rep final: torch.Size([4, 12, 3072])
ac rep final: torch.Size([4, 12, 3072])
fts rep final: torch.Size([4, 12, 3072])
passed and gotten reps for all three successfully!
am rep final in model: torch.Size([4, 12, 3072])
ac rep final in model: torch.Size([4, 12, 3072])
fts rep final in model: torch.Size([4, 12, 3072])
passed first linear
passed second linear
passed third linear


  1%|          | 2/272 [00:01<02:58,  1.51it/s]

...

 14%|█▍        | 39/272 [00:18<01:50,  2.11it/s]

39
am rep final: torch.Size([4, 12, 3072])
ac rep final: torch.Size([4, 12, 3072])
fts rep final: torch.Size([4, 12, 3072])
passed and gotten reps for all three successfully!
am rep final in model: torch.Size([4, 12, 3072])
ac rep final in model: torch.Size([4, 12, 3072])
fts rep final in model: torch.Size([4, 12, 3072])
passed first linear
passed second linear
passed third linear


 15%|█▍        | 40/272 [00:18<01:49,  2.12it/s]


40
am rep final: torch.Size([4, 12, 3072])
ac rep final: torch.Size([4, 12, 3072])
fts rep final: torch.Size([4, 12, 3072])
passed and gotten reps for all three successfully!
am rep final in model: torch.Size([4, 12, 3072])
ac rep final in model: torch.Size([4, 12, 3072])
fts rep final in model: torch.Size([4, 12, 3072])


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

### Predictions

In [ ]:
# # Load best model
# network_2 = Network()

# network_2.load_state_dict(torch.load(path))
# network_2.eval()

In [ ]:
first_model = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
first_model.load_state_dict(torch.load('first_model.pt'))

In [ ]:
model_am = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
model_am.load_state_dict(torch.load('model_am.pt'))

In [ ]:
model_ac = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
model_ac.load_state_dict(torch.load('model_ac.pt'))

In [ ]:
model_fts = BertModel(BertConfig.from_pretrained("bert-base-uncased"))
model_fts.load_state_dict(torch.load('model_fts.pt'))

In [ ]:
# Load best model

custom_model_2 = CustomBERTKuri(first_model, model_am, model_ac, model_fts, 3)
custom_model_2.load_state_dict(torch.load('best_model.pt'))

custom_model_2.to(device).eval()

In [ ]:
def predict(model, test_dataloader=None):
    
    """Prediction loop"""

    preds_l = []
    labels_l = []
    
    model.eval()

    for batch in test_dataloader:            
            
        # unpack batch             
        labels = batch['label'].to(device).flatten().tolist()
        spans = batch['spans'].to(device)
        input_ids = batch['input_ids'].to(device)
        
        inputs = input_ids, spans

        # get output
        
        raw_preds = model(inputs).to('cpu')
        # print(raw_preds.shape)
        raw_preds = raw_preds.detach()#.numpy()

        # Compute argmax
        
        predictions = torch.argmax(raw_preds, dim=2).flatten().tolist()
        preds_l.append(predictions)
        labels_l.append(labels)        
        
        del batch
            
    return flatten_list(preds_l), flatten_list(labels_l)

In [ ]:
#test_preds, test_labels = predict(custom_model, test_dataloader)

In [ ]:
#print(classification_report(test_labels, test_preds, digits=3))

In [ ]:
# remove -100s

In [ ]:
#test_preds_l, test_labels_l = remove_dummy_labels(test_preds, test_labels)

In [ ]:
#print(classification_report(test_labels_l, test_preds_l, digits=3))

In [ ]:
### -100 done!

In [ ]:
test_preds, test_labels = predict(custom_model_2, test_dataloader)

In [ ]:
print(classification_report(test_labels, test_preds, digits=3))

In [ ]:
test_preds_l, test_labels_l = remove_dummy_labels(test_preds, test_labels)

In [ ]:
print(classification_report(test_labels_l, test_preds_l, digits=3))